In [1]:
import numpy as np
class TicTacToe:
    def __init__(self,move=None,state=None,to_play = None):
        #print('Starting Game')
        self.result = None
        if state is None:
            self.state = np.resize(np.zeros(9),(3,3))
        else:
            self.state = state
        self.finished = False
        #print(self.state)
        #Ones start
        if to_play is None:
            self.to_play = 1
        else:
            self.to_play = to_play
        
        #Not sure I want to play random mode at init...
#         if move is None:
#             self.play()
#         else:
#             self.play(move=move)
    
    def board(self):
        print(self.state)
    
    def game_status(self):
        self.result = None
        self.finished = False
            
        for i in range(3):
            #If rows have three in row of same number then someone won
            row_result = self.state[i].sum()
            if row_result in (-3,3):
                self.finished=True
                self.result= 'win' if row_result>0 else 'loss'
            #If columns have three in row of same number then someone won
            column_result = np.rot90(self.state,1)[i].sum()
            if column_result in (-3,3):
                self.finished=True
                self.result= 'win' if column_result>0 else 'loss'
        #if either diagonal have three in row of same number then someone won
        diagonal_result = self.state[0][0]+self.state[1][1]+self.state[2][2]
        if diagonal_result in (-3,3):
            self.finished=True
            self.result= 'win' if diagonal_result>0 else 'loss'
        diagonal_result = self.state[0][2]+self.state[1][1]+self.state[2][0]
        if diagonal_result in (-3,3):
            self.finished=True
            self.result= 'win' if diagonal_result>0 else 'loss'
        
        #If we placed something on all places without a victor then it's a draw
        if 0 not in self.state and self.result is None:
            self.finished=True
            self.result='draw'
        
        
    def play(self,move=None):
        #row_index = int(input('Enter row 0-2: '))
        #column_index = int(input('Enter column 0-2: '))
        row_index = np.random.randint(0,3)
        column_index = np.random.randint(0,3)
        if move is None:
            move = (row_index,column_index)
        
        if row_index>2 or column_index>2:
            #print('out of bounds try again')
            return self.play()
        if self.state[move]!=0:
            #print('someone already played this position, try again')
            return self.play()
        previous_state = self.state
        self.state[move] = self.to_play
        self.move_taken = move
        #print(self.state)
        self.game_status()
        #print('Finished',self.finished)
        if self.finished:
            return#print('Result:',self.result)
        else:
            self.to_play = self.to_play*-1
            #return self.play()

In [2]:
#They have a nodeclass in Muzerro that looks like this
#Not really sure how that works
#So I assume each node has children that are other nodes.
#And each node then have attributes, like score, rewards, who's turn it is to play etc
#THat actually makes a lot of sense!!!!
#So I should be able to run an expansion function on this, to then update the children
#something like this

"""root = Node(0)
    current_observation = game.make_image(-1)
    expand_node(root, game.to_play(), game.legal_actions(),
                network.initial_inference(current_observation))
    add_exploration_noise(config, root)"""
#Note that root node is the first node of the entire treee that has no parents
#A leaf node is a node that doesn't have any children

"""
This process continues, creating a new MCTS tree from scratch each turn and using it to choose an action,
until the end of the game.

So I play an imaginary game from current board position each turn
during those imaginary games I run MCTS for X amount of simulations
It should then return some stuff, that allows me to pick the next turn.
After my next turn, and opponents next turn, I then re-run the MCTS from scratch, not keeping any old historic scores.

"""

class Node(object):

    def __init__(self, prior: float):
        self.visit_count = 0 #Amount of times we have simulated from this node or any of its children
        self.to_play = -1 #Whos turn it is to play, not sure why this isn't part of init parameters?
        self.prior = prior #the predicted prior probability of choosing the action that leads to this node
        self.value_sum = 0 #the backfilled value sum of the node 
        
        #This should be put by an expand_node function if I expand the node, each child being its own node
        #It should be a ditctionary containing the action taken as the key, and the child node that creates as value
        '''visit_counts = [
      (child.visit_count, action) for action, child in node.children.items()
      ]'''
        self.children = {} #{action:childnode}
        
        self.hidden_state = None # the hidden state the node corresponds to
        self.reward = 0 #the predicted reward received by moving to this node

    #This function just lets us know if we have expanded the node already
    def expanded(self) -> bool:
        return len(self.children) > 0

    #This seems to calculate the %ratio a visit of node resulted in win, which should be used for node selection criteria
    #Might also be used as reward for the ML algorithm?
    def value(self) -> float:
        if self.visit_count == 0:
            return 0
        return self.value_sum / self.visit_count

In [3]:
def expand_node(root : Node):
    #This is where we expand our current turn to understand which moves we can make
    moves = []
    current_state = root.hidden_state.copy()
    
    #this gives legal nodes
    #In actual muzero this is done elsewhere I think
    for r in range(0,3):
        for c in range(0,3):
            if current_state[r,c]==0:
                moves.append((r,c))
    #We add a childnode for each possible move
    for move in moves:
        new_state = current_state.copy()
        new_state[move] = root.to_play #The new state of the board if we play this move
        childnode = Node(0) #I create a new childnode, but not sure what to put as prior
        childnode.to_play = root.to_play*-1 #It's the next person's turn at this node
        childnode.hidden_state=new_state #The state of the childnode based on playing this move
        root.children.update({move:childnode}) #Add this node to our possible children nodes        
    #We return the new node now containing its expanded children
    return root


In [4]:
def choose_action(root:Node,legal_moves:list):
    #probably thinking a bit incorrectly here
    #I assume that just the first child selection should be UCB based
    #Next selections should be random?
    #Or maybe not necessairly
    action = None
    #If we havent started yet, then just take random move
    if root.visit_count==0:
        while action not in legal_moves:
            row_index = np.random.randint(0,3)
            column_index = np.random.randint(0,3)
            action = (row_index,column_index)
        return action
    #if we have started then calculate the ucb score
    #for each child node and select highest score
    exploration_values = []
    actions = [] #Will store the actions so we can get child
    for action_key in root.children:
        if root.children[action_key].visit_count==0:
            child_score = float('inf')
        else:
            child_score = ucb_score(parent=root,child=root.children[action_key])
        exploration_values.append(child_score)
        actions.append(action_key)
    highest_value = max(exploration_values)
    action = actions[exploration_values.index(highest_value)]
    return action

In [5]:
import math

In [6]:
def add_exploration_noise(node: Node):
    # Root prior exploration noise.
    root_dirichlet_alpha = 1 #0.3 is for chess, not sure if shoul have lower or higher
    root_exploration_fraction = 0.25
    #add exploration noise at start of search
    actions = list(node.children.keys())
    noise = np.random.dirichlet([root_dirichlet_alpha] * len(actions))
    frac = root_exploration_fraction
    for a, n in zip(actions, noise):
        node.children[a].prior = node.children[a].prior * (1 - frac) + n * frac

In [7]:
def ucb_score(parent: Node, child: Node) -> float:
#     pb_c_base = 19652
#     pb_c_init = 1.25
#     pb_c = math.log((parent.visit_count + pb_c_base + 1) /
#                   pb_c_base) + pb_c_init
#     pb_c *= math.sqrt(parent.visit_count) / (child.visit_count + 1)
#     #print('pc_c',pb_c)
#     prior_score = pb_c * child.prior
#     #print('prior_score',prior_score)
#     value_score = child.value()
#     print('returned_value',prior_score + value_score)
#     return prior_score + value_score
    parent_node_visits = parent.visit_count

    exploration_term = (math.sqrt(2.0)
                        * math.sqrt(math.log(parent_node_visits) / child.visit_count))

    value = child.value() + exploration_term
    return value

In [14]:
def run_shit(root:Node,game:TicTacToe,simulations=1200):
    #This function runs bunch of simulations
    #It then return the move that returned most wins
    #We will try random move 100 times
    for _ in range(simulations):
        #This will play the next move and expand till the end
        fake_game = TicTacToe()
        fake_game.to_play = game.to_play
        fake_game.state = game.state.copy()
        path = [root] #This contains all the nodes we played
        legal_moves = [list(root.children.keys())][0]
        action = choose_action(root,legal_moves)
        current_node = root.children[action]
        while fake_game.result is None:
            path.append(current_node)
            fake_game.play(action)
            if fake_game.result is not None:
                break
            if not current_node.expanded():
                expand_node(current_node)
                add_exploration_noise(current_node)
            legal_moves = [list(current_node.children.keys())][0]
            action = choose_action(current_node,legal_moves)
            #The node corresonding to chosen action
            chosen_node = current_node.children[action]
            current_node = chosen_node

        # we give the result to parent node or all nodes?
        # probably just parent?    
        #back propagate result
        #print(path)
        for node in reversed(path):
            #print(node)
            #print(node.to_play)
            node.visit_count+=1
            #If we win then we get a point
            if fake_game.result=='win' and node.to_play!=fake_game.to_play:
                node.value_sum+=1
            #in tictactoe draw is pretty much expected so we give half a point to all
            if fake_game.result=='draw':
                node.value_sum+=1 #in tictactoe draw is pretty much expected
#                 node.value_sum+=0.5
            #If the other one loses then we get an extra point
            #if fake_game.result=='loss' and node.to_play==fake_game.to_play:
                #node.value_sum+=1
                #print(node.to_play)
                #print(fake_game.state)
#                 node.value_sum-=1

    actions = []
    values = []
    result = {}
    visits = []
    nodes = []
    for action,child in root.children.items():
        nodes.append(child)
        actions.append(action)
        values.append(child.value())
        result.update({action:child.value()})
        visits.append(child.visit_count)
#     return actions[values.index(max(values))],result
    return actions[visits.index(max(visits))],result,visits,nodes

In [15]:
def play_game(turns=2,simulations=5000):
    game = TicTacToe()
    counter = 0
    History=[]
    while game.result is None and counter<turns:
        counter+=1
        if game.to_play==-1:
            root = Node(0) #We create a new root node from scratch
            root.to_play = game.to_play #We give it turn to play based on game's current status
            root.hidden_state = game.state.copy() #We give it the current state of the game
            expand_node(root) #Each time we start new search we expand root
            add_exploration_noise(root) #We then add exploration noise as prior value to children
            move,values,visits,child = run_shit(root,game,simulations) #Simulate some games
            game.play(move)
            History.append([move,values,visits,child,game.state.copy()])
        else:
            row_index = int(input('Enter row 0-2: '))
            column_index = int(input('Enter column 0-2: '))
            move = (row_index,column_index)
            game.play(move)
        print(game.state)
    print(game.result)
    return History

In [16]:
def self_play(number_of_games:int):
    Games = []
    for _ in range(number_of_games): #play some real games to see if it always ties
        History = []
        game = TicTacToe()
        while game.result is None:
            #Initialize a new round of MCTS
            root = Node(0) #We create a new root node from scratch
            root.to_play = game.to_play #We give it turn to play based on game's current status
            root.hidden_state = game.state.copy() #We give it the current state of the game
            expand_node(root) #Each time we start new search we expand root
            add_exploration_noise(root) #We then add exploration noise as prior value to children
            move,values,visits,child = run_shit(root,game) #Simulate some games

            game.play(move)
            History.append([move,values,visits,child,game.state.copy()])
        Games.append([game.result,History])
    return Games

In [45]:
Info = play_game(simulations=6000,turns=2)

Enter row 0-2: 1
Enter column 0-2: 1
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
[[ 0.  0. -1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
None


In [42]:
Info

[[(2, 2),
  {(0, 0): 0.11900532859680284,
   (0, 1): 0.09567198177676538,
   (0, 2): 0.11580882352941177,
   (1, 0): 0.1508313539192399,
   (1, 2): 0.11580882352941177,
   (2, 0): 0.16283716283716285,
   (2, 1): 0.14731585518102372,
   (2, 2): 0.23064770932069512},
  [563, 439, 544, 842, 544, 1001, 801, 1266],
  array([[ 0.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0., -1.]])]]

In [32]:
Interesting3 = Info

In [27]:
Interesting2 = Info

In [22]:
Interesting = Info

In [24]:
Interesting

[[(0, 2),
  {(0, 0): 0.07023411371237458,
   (0, 1): 0.16336633663366337,
   (0, 2): 0.9314027344170525,
   (1, 0): 0.11059907834101383,
   (1, 2): 0.12424849699398798,
   (2, 0): 0.11286681715575621,
   (2, 1): 0.12903225806451613,
   (2, 2): 0.13211009174311927},
  [299, 808, 46445, 434, 499, 443, 527, 545],
  array([[ 0.,  0., -1.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  0.]])]]

In [132]:
Info

[[(1, 2),
  {(0, 0): 0.10044642857142858,
   (0, 1): 0.12769485903814262,
   (0, 2): 0.12050359712230216,
   (1, 0): 0.12413793103448276,
   (1, 2): 0.14675324675324675,
   (2, 0): 0.1432469304229195,
   (2, 1): 0.13916786226685796,
   (2, 2): 0.12887438825448613},
  [448, 603, 556, 580, 770, 733, 697, 613],
  array([[ 0.,  0.,  0.],
         [ 0.,  1., -1.],
         [ 0.,  0.,  0.]])],
 [(2, 2),
  {(0, 1): 0.10810810810810811,
   (0, 2): 0.0,
   (1, 0): 0.0,
   (2, 0): 0.1,
   (2, 1): 0.0,
   (2, 2): 0.7437473505722764},
  [74, 46, 46, 70, 46, 4718],
  array([[ 1.,  0.,  0.],
         [ 0.,  1., -1.],
         [ 0.,  0., -1.]])],
 [(0, 1),
  {(0, 1): 0.6593373493975904,
   (1, 0): 0.0,
   (2, 0): 0.6280940594059405,
   (2, 1): 0.0},
  [3320, 32, 1616, 32],
  array([[ 1., -1.,  1.],
         [ 0.,  1., -1.],
         [ 0.,  0., -1.]])]]

In [128]:
Info

[[(0, 2),
  {(0, 0): 0.15102639296187684,
   (0, 1): 0.15160349854227406,
   (0, 2): 0.16974169741697417,
   (1, 0): 0.1308411214953271,
   (1, 2): 0.1504424778761062,
   (2, 0): 0.14947683109118087,
   (2, 1): 0.11409395973154363,
   (2, 2): 0.12244897959183673},
  [682, 686, 813, 535, 678, 669, 447, 490],
  array([[ 0.,  0., -1.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  0.]])],
 [(0, 0),
  {(0, 0): 0.9186454676853191,
   (0, 1): 0.0,
   (1, 0): 0.07142857142857142,
   (1, 2): 0.12121212121212122,
   (2, 0): 0.0,
   (2, 1): 0.24},
  [4843, 23, 28, 33, 23, 50],
  array([[-1.,  0., -1.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  1.]])],
 [(2, 1),
  {(1, 0): 0.53125,
   (1, 2): 0.5925925925925926,
   (2, 0): 0.0,
   (2, 1): 0.9946269890473238},
  [64, 81, 16, 4839],
  array([[-1.,  1., -1.],
         [ 0.,  1.,  0.],
         [ 0., -1.,  1.]])],
 [(1, 2),
  {(1, 2): 1.0, (2, 0): 0.0},
  [4984, 16],
  [<__main__.Node at 0x2bb345576a0>, <__main__.Node at 0x2bb34557710>],
  arra

In [111]:
Info

[[(1, 0),
  {(0, 0): 0.11029411764705882,
   (0, 1): 0.11347517730496454,
   (0, 2): 0.1263318112633181,
   (1, 0): 0.15416666666666667,
   (1, 2): 0.10412573673870335,
   (2, 0): 0.10687022900763359,
   (2, 1): 0.13914174252275682,
   (2, 2): 0.09725158562367865},
  [544, 564, 657, 960, 509, 524, 769, 473],
  array([[ 0.,  0.,  0.],
         [-1.,  1.,  0.],
         [ 0.,  0.,  0.]])]]

In [94]:
test = [0,1,2]

In [95]:
for item in reversed(test):
    print(item)

2
1
0


In [86]:
Info

[[(0, 1),
  {(0, 1): 1.0,
   (0, 2): 1.0,
   (1, 0): 1.0,
   (1, 1): 1.0,
   (1, 2): 1.0,
   (2, 0): 1.0,
   (2, 1): 1.0,
   (2, 2): 1.0},
  [625, 625, 625, 625, 625, 625, 625, 625],
  array([[ 1., -1.,  0.],
         [ 0.,  0.,  0.],
         [ 0.,  0.,  0.]])]]

In [48]:
Info[0][3][4].visit_count

659

In [49]:
Info[0][3][4].value()

0.061456752655538696

In [50]:
Info[0][3][4].value_sum

40.5

In [53]:
#This should have lead to Draw? Why only 39 points
#Out of 648
Info[0][3][0].visit_count,Info[0][3][0].value_sum

(648, 39.0)

In [61]:
weird = Info[0][3][0]
for action_key in weird.children:
    print(weird.children[action_key].hidden_state)
    print(weird.children[action_key].visit_count)
    print(weird.children[action_key].value_sum)

[[-1.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
115
89.5
[[-1.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
122
96.0
[[-1.  0.  0.]
 [ 1.  1.  0.]
 [ 0.  0.  0.]]
107
82.5
[[-1.  0.  0.]
 [ 0.  1.  1.]
 [ 0.  0.  0.]]
90
66.5
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]]
83
60.0
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]]
85
61.5
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
46
27.0


In [64]:
new.to_play

-1

In [65]:
weird.to_play

1

In [63]:
new = weird.children[(0,2)]
for action_key in new.children:
    print(new.children[action_key].hidden_state)
    print(new.children[action_key].visit_count)
    print(new.children[action_key].value_sum)

[[-1. -1.  1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
23
2.0
[[-1.  0.  1.]
 [-1.  1.  0.]
 [ 0.  0.  0.]]
18
0
[[-1.  0.  1.]
 [ 0.  1. -1.]
 [ 0.  0.  0.]]
23
2.0
[[-1.  0.  1.]
 [ 0.  1.  0.]
 [-1.  0.  0.]]
21
1.5
[[-1.  0.  1.]
 [ 0.  1.  0.]
 [ 0. -1.  0.]]
19
0.5
[[-1.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]
18
0


In [59]:
weird = Info[0][3][4]
for action_key in weird.children:
    print(weird.children[action_key].visit_count)
    print(weird.children[action_key].value_sum)

97
79.0
79
60.5
114
95.5
62
44.5
88
70.0
101
82.0
118
100.0


In [34]:
testing = self_play(10)

In [35]:
for result,hisotry in testing:
    print(result)

draw
draw
draw
draw
draw
draw
draw
draw
draw
draw


In [26]:
Info

[[(0, 1),
  {(0, 0): 0.10152284263959391,
   (0, 1): 0.16037735849056603,
   (0, 2): 0.11946902654867257,
   (1, 0): 0.09844559585492228,
   (1, 2): 0.15202702702702703,
   (2, 0): 0.10396039603960396,
   (2, 1): 0.1551155115511551,
   (2, 2): 0.13962264150943396},
  [197, 318, 226, 193, 296, 202, 303, 265],
  array([[ 0., -1.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  0.]])]]

In [ ]:
for result,history in Games:
    print(result)

In [ ]:
History #Seems like it doesnt really work as expected
#I keep only visiting high value, so exploration noise needs to be higher?
#or something else is wrong?

In [ ]:
game.result

In [ ]:
SavedGame = History

In [ ]:
Test = History

In [ ]:
Test==SavedGame

In [ ]:
SavedGame

In [ ]:
Test

In [ ]:
History[3][3][0].visit_count

In [ ]:
weird = History[3][3][0]
for action in weird.children:
    print(weird.children[action].visit_count)
    print(weird.children[action].value())
    print(weird.children[action].to_play)
    print(weird.children[action].hidden_state)

In [ ]:
weird.to_play

In [ ]:
root.visit_count

In [ ]:
for key in root.children:
    print(root.children[key].visit_count)
    print(root.children[key].prior)
    print(root.children[key].value())

In [ ]:
add_exploration_noise(root)

In [ ]:
root.children[(0,0)].prior

In [ ]:
current_node.hidden_state

In [ ]:
root.visit_count

In [ ]:
root.children[(0,0)].visit_count

In [ ]:
root.visit_count

In [ ]:
root.value()

In [ ]:
game.state

In [ ]:

for state,move,values in History:
    print(state,move,values)

In [ ]:
#How do I make it understand that not playing 
#(0,2) here means loss
[[ 0.  0.  0.]
 [ 0.  1.  0.]
 [ 1. -1.  0.]]

#How do I make it understand that playing
#(0,2) here is the best win, even though 0,0 also keeps 100% win?
[[ 0. -1.  0.]
 [ 0.  1.  0.]
 [ 1. -1.  0.]]

In [ ]:
game.state

In [ ]:
root.children

In [ ]:
actions = []
values = []
for action,child in root.children.items():
    actions.append(action)
    values.append(child.value())
values

In [ ]:
values

In [ ]:
root.value_sum

In [ ]:
choose_best_action(root)

In [ ]:
parent.value_sum

In [ ]:
parent.visit_count

In [ ]:
fake_game.result

In [ ]:
path[0].hidden_state

In [ ]:
fake_game.state

In [ ]:
for node in path:
    print(node.hidden_state)

In [ ]:
current_node.hidden_state

In [ ]:
#     counter = 1
#     for turn in stats_tree:
#         counter+=1
    #A big problem here is that turn_2 can obviously look very different!!!!
    #so I need to have a different turn 2 for each possibility
    #the new method of children might work, but also need to traverse the depth of them somehow.
    root.children.update(action)
    
    stats_tree.update({'turn_'+str(counter):{}})
    for move in moves:
        stats_tree['turn_'+str(counter)].update({move:[0,0]})
    
    return stats_tree

In [ ]:
#stats_tree = {'turn_1':{}}
stats_tree = {}
for move in moves:
    stats_tree[move] = {'children':{},'score':[0,0]}

In [ ]:
for _ in range(1):
    move = selection(stats_tree)
    stats_tree = update(stats_tree,move,np.resize(np.zeros(9),(3,3)))

#state = np.resize(np.zeros(9),(3,3))
#state[0,0]=1
#state[0,1]=-1
                  
#stats_tree = expansion(state,stats_tree)

In [ ]:
stats_tree

In [ ]:
moves

In [ ]:
stats_tree[(0,0)]

In [ ]:
stats_tree = {}
for move in moves:
    stats_tree[move] = {'children':{},'score':[1,1]}

In [ ]:
stats_tree

In [ ]:
stats_tree['turn_1'][(0, 0)] = {'children':{},'score':[1,1]}

In [ ]:
stats_tree

In [ ]:
stats_tree['turn_1'][(0, 0)]

In [ ]:
simulation((0,0),np.resize(np.zeros(9),(3,3)))

In [ ]:
expansion(np.resize(np.zeros(9),(3,3)))

In [ ]:
expansion(np.identity(3))

In [ ]:
results

In [ ]:
moves.pop(moves.index((0,1)))

In [ ]:
moves

In [ ]:
#Pick random move? But I shouldnt really pick random one?
np.random.randint(0,len(moves))

In [ ]:
game = TicTacToe()

In [ ]:
game = TicTacToe()